In [2]:
import pywt
import os
from pylab import *
from numpy import*
import pandas as pd

#### Обработка графиков ЭКГ и преобразование в таблицу

In [3]:
def get_ecg(key):
  way = 'train/' + key + '.npy'
  ecgsig = np.load(way).transpose().reshape((60000,))
  x = np.linspace(0,  1,  num = 2048)
  data = np.sin(250 * np.pi * x**2)
  coefs = pywt.downcoef('a', data, 'db20', mode='symmetric', level=1)
  st='sym5'
  coeffs = pywt.wavedec(ecgsig, st, level=5)
  row = coeffs[0]
  return row

In [4]:
def into_one_table(keys):
  row = get_ecg(keys[0])
  row = np.insert(row, 0, int(keys[0].strip('_hr')))
  table = [row]
  for i in range(target.index.start + 1, target.index.stop):
    row = get_ecg(keys[i])
    row = np.insert(row, 0, int(keys[i].strip('_hr')))
    table = np.vstack([table, row])
  table = pd.DataFrame(table)
  return table

In [5]:
target = pd.Series(list(map(lambda x: x[:-4], os.listdir('train')[:-2])))
ECG_data = into_one_table(target)

In [6]:
ECG_data[0] = ECG_data[0].astype('int')
ECG_data = ECG_data.rename(columns={0: 'ecg_id'})

In [7]:
ECG_data = ECG_data.sort_values(by=['ecg_id'], ascending=True)

In [8]:
ECG_data

,ecg_id,1,2,3,4,5,6,7,8,9,...,1874,1875,1876,1877,1878,1879,1880,1881,1882,1883
0,9,-0.170442,-0.066933,-0.140079,-0.478502,-1.245167,-0.979738,-0.738943,-0.476043,-0.349288,...,0.558936,0.551860,0.551007,0.561618,0.556675,0.645774,0.744957,0.649421,0.667106,0.731759
1,34,0.399648,0.579593,0.460080,-0.077166,-1.270282,-1.040847,-1.689153,-2.481032,-2.205554,...,1.113606,1.088111,1.099680,1.132341,1.130250,1.788834,2.455783,1.818630,1.938019,2.363243
2,43,0.411589,0.365335,0.399006,0.554728,0.911140,0.774487,0.576258,0.404740,0.229235,...,0.341701,0.340029,0.342162,0.341734,0.340499,0.346990,0.366654,0.343959,0.348800,0.365320
3,52,-0.162251,-0.159025,-0.159327,-0.163111,-0.158814,-0.159398,-0.162262,-0.174764,-0.179583,...,-0.267047,-0.258175,-0.261318,-0.274144,-0.273755,-0.519687,-0.765086,-0.532849,-0.575938,-0.730437
4,57,-0.268330,-0.279812,-0.270770,-0.232069,-0.136626,-0.155710,-0.117283,-0.096148,-0.074360,...,-0.304742,-0.297821,-0.302716,-0.310384,-0.311176,-0.513279,-0.714933,-0.521431,-0.558040,-0.686983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,21784,-0.094946,-0.097736,-0.095932,-0.087848,-0.070772,-0.076037,-0.082161,-0.089443,-0.092388,...,0.005842,0.004748,0.005003,0.006885,0.006829,0.041791,0.079657,0.044349,0.050701,0.074361
2097,21795,-0.194416,-0.182553,-0.191956,-0.238293,-0.352612,-0.314071,-0.319038,-0.313175,-0.285890,...,0.120157,0.117738,0.121368,0.121892,0.122540,0.189928,0.260724,0.190226,0.204078,0.251573
2098,21825,-0.004072,-0.000198,-0.003268,-0.017719,-0.054790,-0.048271,-0.088630,-0.084493,-0.070101,...,-0.128999,-0.124998,-0.127924,-0.131497,-0.131004,-0.222249,-0.318699,-0.224482,-0.242503,-0.305902
2099,21831,-0.128454,-0.127641,-0.128944,-0.136319,-0.158699,-0.147968,-0.147763,-0.142817,-0.138197,...,-0.159457,-0.155589,-0.158779,-0.161389,-0.160510,-0.233738,-0.313892,-0.233884,-0.249503,-0.303710


## N-граммы

In [9]:
import neurokit2
import sleepecg
import wfdb.processing
from nltk.util import ngrams

from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

In [10]:
def find_peaks(ecgsignal, peak):
    return wfdb.processing.correct_peaks(ecgsignal, rpeaks,
            search_radius=36, smooth_window_size=50, peak_dir=peak)

In [11]:
def get_cycle(ecgsignal, rpeaks): # разбиение ЭКГ на циклы
  ecgsignal_split = []
  cycle = []
  rpeaks = rpeaks[::-1]
  f = True #флаг конца массива с пиками
  f1 = True #флаг вхождения первого пика
  for i in range(len(ecgsignal)):
    if f and i == rpeaks[-1]:
      if f1:
        f1 = False
        cycle = []
      else:
        ecgsignal_split.append(cycle)
        cycle = []
      rpeaks = rpeaks[:-1]
      if f and len(rpeaks) == 0:
        f = False
    cycle.append(ecgsignal[i])
  return ecgsignal_split

In [12]:
def sign(x):
   return 1 if x > 0 else -1

In [13]:
def RTL(ecgsignal_split):
    sp = [[(i[0] - min(i[:10])), len(i), ((i[0] - min(i[:10]))/len(i))] for i in ecgsignal_split]
    s = []
    for i in range(len(sp) - 1):
        R = sp[i + 1][0] - sp[i][0]
        T = sp[i + 1][1] - sp[i][1]
        L = sp[i + 1][2] - sp[i][2]
        s.append([R, T, L])
    return s

In [14]:
def letters(data): # кодирование цикла по В.М. Успенскому 
  if data[0] == 1:
    if data[1] == -1:
      return 'C'
    else:
      if data[2] == -1:
        return 'E'
      return 'A'
  else:
    if data[1] == 1:
      return 'D'
    else:
      if data[2] == 1:
        return 'B'
      return 'F'

In [15]:
def encoding(ecgsignal_split):
    s = [[sign(i) for i in k] for k in RTL(ecgsignal_split)]
    stroka = ''
    for i in s:
        stroka += letters(i)
    return stroka

In [16]:
ECG_data_array = np.array(ECG_data)

In [17]:
ecg_ids = []
encoding_ecgs = []

In [18]:
for ecgsignal in ECG_data_array:
    ecg_id = ecgsignal[0]
    ecgsignal = ecgsignal[1:]
    
    _, results = neurokit2.ecg_peaks(ecgsignal, sampling_rate=100)
    rpeaks = results["ECG_R_Peaks"]
    rpeaks = sleepecg.detect_heartbeats(ecgsignal, fs=100)
    rpeaks = wfdb.processing.xqrs_detect(ecgsignal, fs=100, verbose=False)
    
    rpeaks = find_peaks(ecgsignal, 'up')
    
    ecgsignal_split = get_cycle(ecgsignal, rpeaks)
    encoding_ecg = encoding(ecgsignal_split)
    
    ecg_ids.append(ecg_id)
    encoding_ecgs.append(encoding_ecg)

In [19]:
encoding_ecg = pd.Series(encoding_ecgs, index=ecg_ids)

In [20]:
gts = pd.read_csv('train/train_gts.csv')
gts.record_name = gts.record_name.map(lambda x: float(x[:-3]))
gts = gts.sort_values(by='record_name')
gts.insert(2, 'encoding_ecg', array(encoding_ecg))
gts

,record_name,myocard,encoding_ecg
265,9.0,0,DDCADDCC
1407,34.0,0,DDDCCCFFFDDFCCCACFE
1833,43.0,0,BFABCFADDEC
441,52.0,0,DDEFBCCBCCF
1834,57.0,0,DDCADBCDD
...,...,...,...
1753,21784.0,0,DDCCFFAFAFFAFC
2008,21795.0,0,
72,21825.0,0,ECCDDDECB
2100,21831.0,0,DAFCCDDDCFCD


In [21]:
sick = gts[gts.myocard == 1].encoding_ecg
healthy = gts[gts.myocard == 0].encoding_ecg

In [22]:
sp_sick= []
for i in sick:
    n = list(map(lambda x: ''.join(list(x)), list(ngrams(list(i), 6))))
    for k in n:
        sp_sick.append(k)

In [23]:
sp_healthy= []
for i in healthy:
    n = list(map(lambda x: ''.join(list(x)), list(ngrams(list(i), 6))))
    for k in n:
        sp_healthy.append(k) 

In [24]:
def counter_and_sort(arr):
  quantity = {}
  for i in arr:
    if i in quantity:
      quantity[i] += 1
    else:
      quantity[i] = 1
  quantity = pd.Series(list(quantity.values()), index=list(quantity.keys())).sort_values(ascending=False)
  return quantity

In [25]:
q_sick = counter_and_sort(sp_sick)
q_health = counter_and_sort(sp_healthy)

In [26]:
only_sick = list(set(q_sick.index) - set(q_health.index))
only_health = list(set(q_health.index) - set(q_sick.index))

In [27]:
popular_sick = q_sick.iloc[:len(q_sick) // 2]
popular_health = q_health.iloc[:len(q_health) // 2]

In [28]:
popular_sick_only = list(set(popular_sick.index) - set(popular_health.index))
popular_health_only = list(set(popular_health.index) - set(popular_sick.index))

In [29]:
ngramms_ecgs = []
for ecg in array(gts):
    ngramms_ecg = set()
    n = list(map(lambda x: ''.join(list(x)), list(ngrams(list(ecg[2]), 6))))
    for k in n:
        ngramms_ecg.add(k)
    ngramms_ecgs.append(list(ngramms_ecg))

In [30]:
def func(ng):
    ng_health = sum(list(map(lambda x: 1 if x in only_health else 0, ng)))
    ng_sick = sum(list(map(lambda x: 1 if x in only_sick else 0, ng)))
    
    ng_popular_health = sum(list(map(lambda x: 1 if x in popular_health_only else 0, ng)))
    ng_popular_sick = sum(list(map(lambda x: 1 if x in popular_sick_only else 0, ng)))
    
    return [ng_health, ng_sick, ng_popular_health, ng_popular_sick]

In [31]:
arr = pd.DataFrame([func(ng) for ng in ngramms_ecgs])

In [32]:
X_train, X_test, y_train, y_test = train_test_split(arr, gts.myocard, random_state=0, test_size=0.75)

In [33]:
cat = CatBoostClassifier(depth=9,  learning_rate=0.03)
cat.fit(X_train, y_train)

0:	learn: 0.6444023	total: 150ms	remaining: 2m 30s
1:	learn: 0.6138423	total: 154ms	remaining: 1m 16s
2:	learn: 0.5774073	total: 159ms	remaining: 52.8s
3:	learn: 0.5306864	total: 161ms	remaining: 40.1s
4:	learn: 0.4964606	total: 164ms	remaining: 32.6s
5:	learn: 0.4700599	total: 166ms	remaining: 27.6s
6:	learn: 0.4540542	total: 168ms	remaining: 23.9s
7:	learn: 0.4295647	total: 170ms	remaining: 21.1s
8:	learn: 0.4147643	total: 172ms	remaining: 18.9s
9:	learn: 0.3949265	total: 174ms	remaining: 17.2s
10:	learn: 0.3697582	total: 175ms	remaining: 15.7s
11:	learn: 0.3527951	total: 177ms	remaining: 14.6s
12:	learn: 0.3357232	total: 179ms	remaining: 13.6s
13:	learn: 0.3178687	total: 181ms	remaining: 12.8s
14:	learn: 0.3044209	total: 183ms	remaining: 12s
15:	learn: 0.2972470	total: 185ms	remaining: 11.4s
16:	learn: 0.2931901	total: 187ms	remaining: 10.8s
17:	learn: 0.2835730	total: 189ms	remaining: 10.3s
18:	learn: 0.2761427	total: 191ms	remaining: 9.87s
19:	learn: 0.2699636	total: 193ms	remaini

230:	learn: 0.1643746	total: 529ms	remaining: 1.76s
231:	learn: 0.1643580	total: 531ms	remaining: 1.76s
232:	learn: 0.1642912	total: 532ms	remaining: 1.75s
233:	learn: 0.1642774	total: 534ms	remaining: 1.75s
234:	learn: 0.1642709	total: 535ms	remaining: 1.74s
235:	learn: 0.1642189	total: 537ms	remaining: 1.74s
236:	learn: 0.1642073	total: 539ms	remaining: 1.74s
237:	learn: 0.1641763	total: 541ms	remaining: 1.73s
238:	learn: 0.1641119	total: 543ms	remaining: 1.73s
239:	learn: 0.1640585	total: 545ms	remaining: 1.73s
240:	learn: 0.1640436	total: 547ms	remaining: 1.72s
241:	learn: 0.1639729	total: 548ms	remaining: 1.72s
242:	learn: 0.1639566	total: 550ms	remaining: 1.71s
243:	learn: 0.1639437	total: 552ms	remaining: 1.71s
244:	learn: 0.1639322	total: 554ms	remaining: 1.71s
245:	learn: 0.1638655	total: 556ms	remaining: 1.7s
246:	learn: 0.1638000	total: 558ms	remaining: 1.7s
247:	learn: 0.1637765	total: 559ms	remaining: 1.7s
248:	learn: 0.1637167	total: 561ms	remaining: 1.69s
249:	learn: 0.1

462:	learn: 0.1604132	total: 926ms	remaining: 1.07s
463:	learn: 0.1604093	total: 929ms	remaining: 1.07s
464:	learn: 0.1604054	total: 931ms	remaining: 1.07s
465:	learn: 0.1604010	total: 933ms	remaining: 1.07s
466:	learn: 0.1603976	total: 935ms	remaining: 1.07s
467:	learn: 0.1603937	total: 937ms	remaining: 1.06s
468:	learn: 0.1603900	total: 939ms	remaining: 1.06s
469:	learn: 0.1603875	total: 941ms	remaining: 1.06s
470:	learn: 0.1603842	total: 943ms	remaining: 1.06s
471:	learn: 0.1603806	total: 944ms	remaining: 1.06s
472:	learn: 0.1603771	total: 946ms	remaining: 1.05s
473:	learn: 0.1603725	total: 947ms	remaining: 1.05s
474:	learn: 0.1603693	total: 949ms	remaining: 1.05s
475:	learn: 0.1603656	total: 951ms	remaining: 1.05s
476:	learn: 0.1603620	total: 952ms	remaining: 1.04s
477:	learn: 0.1603582	total: 954ms	remaining: 1.04s
478:	learn: 0.1603550	total: 956ms	remaining: 1.04s
479:	learn: 0.1603517	total: 957ms	remaining: 1.04s
480:	learn: 0.1603482	total: 959ms	remaining: 1.03s
481:	learn: 

675:	learn: 0.1599334	total: 1.31s	remaining: 630ms
676:	learn: 0.1599322	total: 1.31s	remaining: 628ms
677:	learn: 0.1599311	total: 1.32s	remaining: 626ms
678:	learn: 0.1599297	total: 1.32s	remaining: 624ms
679:	learn: 0.1599286	total: 1.32s	remaining: 622ms
680:	learn: 0.1599272	total: 1.32s	remaining: 620ms
681:	learn: 0.1599258	total: 1.34s	remaining: 625ms
682:	learn: 0.1599245	total: 1.34s	remaining: 623ms
683:	learn: 0.1599232	total: 1.34s	remaining: 621ms
684:	learn: 0.1599219	total: 1.35s	remaining: 620ms
685:	learn: 0.1599206	total: 1.35s	remaining: 618ms
686:	learn: 0.1599193	total: 1.35s	remaining: 616ms
687:	learn: 0.1599182	total: 1.35s	remaining: 613ms
688:	learn: 0.1599169	total: 1.35s	remaining: 612ms
689:	learn: 0.1599157	total: 1.36s	remaining: 609ms
690:	learn: 0.1599145	total: 1.36s	remaining: 608ms
691:	learn: 0.1599133	total: 1.36s	remaining: 606ms
692:	learn: 0.1599119	total: 1.36s	remaining: 604ms
693:	learn: 0.1599108	total: 1.36s	remaining: 602ms
694:	learn: 

886:	learn: 0.1597350	total: 1.71s	remaining: 218ms
887:	learn: 0.1597344	total: 1.71s	remaining: 216ms
888:	learn: 0.1597335	total: 1.71s	remaining: 214ms
889:	learn: 0.1597328	total: 1.72s	remaining: 212ms
890:	learn: 0.1597321	total: 1.72s	remaining: 210ms
891:	learn: 0.1597314	total: 1.72s	remaining: 208ms
892:	learn: 0.1597308	total: 1.72s	remaining: 206ms
893:	learn: 0.1597302	total: 1.72s	remaining: 204ms
894:	learn: 0.1597295	total: 1.72s	remaining: 202ms
895:	learn: 0.1597289	total: 1.73s	remaining: 200ms
896:	learn: 0.1597282	total: 1.73s	remaining: 198ms
897:	learn: 0.1597274	total: 1.73s	remaining: 196ms
898:	learn: 0.1597269	total: 1.73s	remaining: 194ms
899:	learn: 0.1597262	total: 1.73s	remaining: 193ms
900:	learn: 0.1597255	total: 1.73s	remaining: 191ms
901:	learn: 0.1597250	total: 1.74s	remaining: 189ms
902:	learn: 0.1597242	total: 1.74s	remaining: 187ms
903:	learn: 0.1597237	total: 1.74s	remaining: 185ms
904:	learn: 0.1597230	total: 1.74s	remaining: 183ms
905:	learn: 

In [59]:
f1_score(y_test, cat.predict(X_test),  average = 'macro')

0.8612836391384646

In [58]:
f1_score(y_train, cat.predict(X_train), average = 'macro')

0.8722361284939507